In [4]:
%pip install transformers[torch]==4.45.2 huggingface_hub datasets evaluate torchvision

  Using cached transformers-4.45.2-py3-none-any.whl (9.9 MB)
  Using cached huggingface_hub-0.26.3-py3-none-any.whl (447 kB)
  Using cached datasets-3.1.0-py3-none-any.whl (480 kB)
  Using cached evaluate-0.4.3-py3-none-any.whl (84 kB)
  Using cached torchvision-0.20.1-cp310-cp310-manylinux1_x86_64.whl (7.2 MB)
  Using cached tokenizers-0.20.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Using cached PyYAML-6.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (751 kB)
  Using cached safetensors-0.4.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (435 kB)
  Using cached filelock-3.16.1-py3-none-any.whl (16 kB)
  Using cached regex-2024.11.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (781 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 2.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.0 MB/s eta 0:00:0000:0100:21
 

In [27]:
from transformers import MobileNetV2Config, MobileNetV2ForImageClassification, AutoModelForImageClassification, Trainer, TrainingArguments
from torchvision import transforms, datasets as dataset
from torch.utils.data import Dataset
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import evaluate
import random
import torch
import os

In [ ]:
def reset_seed(seed=42):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) 
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

In [ ]:
class CIFAR10HFDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        return {
            'pixel_values': image,
            'labels': label
        }

    def __len__(self):
        return len(self.dataset)

In [7]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    acc = accuracy.compute(references=labels, predictions=np.argmax(predictions, axis=1))
    return {"accuracy": acc["accuracy"]}

In [8]:
def get_training_args(output_dir, logging_dir):
    return (
        TrainingArguments(
        output_dir=output_dir,
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=5e-5,
        per_device_train_batch_size=64,
        per_device_eval_batch_size=64,
        num_train_epochs=20,
        weight_decay=0.01,
        seed = 42,
        metric_for_best_model="accuracy",
        load_best_model_at_end=True,
        fp16=True, 
        logging_dir=logging_dir,
    ))


In [ ]:
def get_random_init_mobilenet():
    student_config = MobileNetV2Config()
    student_config.num_labels = 10
    return MobileNetV2ForImageClassification(student_config)

In [ ]:
def freeze_model(model):
    for param in model.parameters():
        param.requires_grad = False

    for param in model.classifier.parameters():
        param.requires_grad = True

In [ ]:
def get_mobilenet():
    model_pretrained = MobileNetV2ForImageClassification.from_pretrained("google/mobilenet_v2_1.0_224")
    in_features = model_pretrained.classifier.in_features

    model_pretrained.classifier = nn.Linear(in_features,10)
    model_pretrained.num_labels = 10
    model_pretrained.config.num_labels = 10

    return model_pretrained

In [ ]:
reset_seed(42)

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])


train = dataset.CIFAR10(root='./data/10', train=True, download=True, transform=transform)
test = dataset.CIFAR10(root='./data/10', train=False, download=True, transform=transform)

train_dataset_hf = CIFAR10HFDataset(train)
test_dataset_hf = CIFAR10HFDataset(test)

In [ ]:
training_args = get_training_args("./results/cifar10-random", './logs/cifar10-random')
model = get_random_init_mobilenet()

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_hf,
    eval_dataset=test_dataset_hf,
    compute_metrics=compute_metrics,
)

In [11]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.999400,1.465344,0.461900
2,1.338000,1.238446,0.563300
3,1.135700,1.040728,0.626200
4,0.888400,0.949196,0.677800
5,0.773600,0.940272,0.678500
6,0.618400,0.835595,0.726800
7,0.558500,1.199692,0.655300
8,0.431500,1.572898,0.602600
9,0.356000,0.850076,0.743500
10,0.288900,0.897842,0.753100


TrainOutput(global_step=15640, training_loss=0.45282880482466326, metrics={'train_runtime': 3560.2383, 'train_samples_per_second': 280.88, 'train_steps_per_second': 4.393, 'total_flos': 2.020099608576e+18, 'train_loss': 0.45282880482466326, 'epoch': 20.0})

In [11]:
model.eval()

MobileNetV2ForImageClassification(
  (mobilenet_v2): MobileNetV2Model(
    (conv_stem): MobileNetV2Stem(
      (first_conv): MobileNetV2ConvLayer(
        (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (conv_3x3): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (reduce_1x1): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (normalization): BatchNorm2d(16, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
      )
    )
    (layer): ModuleList(
      (0): MobileNetV2InvertedResidual(
        (expand_1x1): MobileNe

In [12]:
trainer.evaluate()

{'eval_loss': 1.0349912643432617,
 'eval_accuracy': 0.7594,
 'eval_runtime': 12.9818,
 'eval_samples_per_second': 770.311,
 'eval_steps_per_second': 12.094,
 'epoch': 20.0}

In [32]:
class ImageDistilTrainer(Trainer):
    def __init__(self, teacher_model=None, student_model=None, temperature=None, lambda_param=None, *args, **kwargs):
        super().__init__(model=student_model, *args, **kwargs)
        self.teacher = teacher_model
        self.student = student_model
        self.loss_function = nn.KLDivLoss(reduction="batchmean")
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.teacher.to(device)
        self.teacher.eval()
        self.temperature = temperature
        self.lambda_param = lambda_param



    def compute_loss(self, student, inputs, return_outputs=False):
        student_output = self.student(**inputs)

        with torch.no_grad():
          teacher_output = self.teacher(**inputs)


        soft_teacher = F.softmax(teacher_output.logits / self.temperature, dim=-1)
        soft_student = F.log_softmax(student_output.logits / self.temperature, dim=-1)

        
        distillation_loss = self.loss_function(soft_student, soft_teacher) * (self.temperature ** 2)

 
        student_target_loss = student_output.loss


        loss = (1. - self.lambda_param) * student_target_loss + self.lambda_param * distillation_loss
        return (loss, student_output) if return_outputs else loss

In [42]:
reset_seed(42)

In [43]:
teacher_model = AutoModelForImageClassification.from_pretrained(
    "aaraki/vit-base-patch16-224-in21k-finetuned-cifar10",
    num_labels=10
)

student_model = get_random_init_mobilenet()

In [ ]:
training_args = get_training_args("./results/cifar10-random-KD", './logs/cifar10-random-KD')

In [46]:
trainer = ImageDistilTrainer(
    student_model=student_model,
    teacher_model=teacher_model,
    args=training_args,
    train_dataset=train_dataset_hf,
    eval_dataset=test_dataset_hf,
    compute_metrics=compute_metrics,
    temperature = 5,
    lambda_param = 0.6
)

In [47]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.379100,1.075377,0.414800
2,0.967200,0.870000,0.555300
3,0.836300,0.705798,0.648300
4,0.666400,0.651311,0.682100
5,0.589900,0.837380,0.634500
6,0.497600,0.562861,0.727100
7,0.460800,0.651885,0.702000
8,0.391900,0.790115,0.625500
9,0.351200,0.604098,0.717100
10,0.314400,0.544215,0.758300


TrainOutput(global_step=15640, training_loss=0.4183128375226579, metrics={'train_runtime': 24817.0423, 'train_samples_per_second': 40.295, 'train_steps_per_second': 0.63, 'total_flos': 2.020099608576e+18, 'train_loss': 0.4183128375226579, 'epoch': 20.0})

In [ ]:
student_model.eval()

MobileNetV2ForImageClassification(
  (mobilenet_v2): MobileNetV2Model(
    (conv_stem): MobileNetV2Stem(
      (first_conv): MobileNetV2ConvLayer(
        (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (conv_3x3): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (reduce_1x1): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (normalization): BatchNorm2d(16, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
      )
    )
    (layer): ModuleList(
      (0): MobileNetV2InvertedResidual(
        (expand_1x1): MobileNe

In [ ]:
trainer.evaluate()

{'eval_loss': 0.4978441298007965,
 'eval_accuracy': 0.7746,
 'eval_runtime': 91.479,
 'eval_samples_per_second': 109.315,
 'eval_steps_per_second': 1.716,
 'epoch': 20.0}

In [17]:
reset_seed(42)

In [18]:
model_pretrained = get_mobilenet()

In [19]:
print(model_pretrained)

MobileNetV2ForImageClassification(
  (mobilenet_v2): MobileNetV2Model(
    (conv_stem): MobileNetV2Stem(
      (first_conv): MobileNetV2ConvLayer(
        (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (conv_3x3): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (reduce_1x1): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (normalization): BatchNorm2d(16, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
      )
    )
    (layer): ModuleList(
      (0): MobileNetV2InvertedResidual(
        (expand_1x1): MobileNe

In [ ]:
freeze_model(model_pretrained)

In [21]:
training_args = get_training_args("./results/cifar10-pretrained-head", './logs/cifar10-pretrained-head')

In [22]:
trainer = Trainer(
    model=model_pretrained,
    args=training_args,
    train_dataset=train_dataset_hf,
    eval_dataset=test_dataset_hf,
    compute_metrics=compute_metrics,
)

In [23]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.890400,1.359479,0.631100
2,1.122600,1.207260,0.624200
3,1.016800,0.971185,0.709800
4,0.904100,0.907877,0.715000
5,0.876800,0.984696,0.681200
6,0.832400,0.870024,0.722000
7,0.827400,1.021015,0.655300
8,0.802100,1.105050,0.629500
9,0.791900,0.866775,0.707400
10,0.785000,0.875023,0.712700


TrainOutput(global_step=15640, training_loss=0.8626149672681414, metrics={'train_runtime': 1668.8023, 'train_samples_per_second': 599.232, 'train_steps_per_second': 9.372, 'total_flos': 2.020099608576e+18, 'train_loss': 0.8626149672681414, 'epoch': 20.0})

In [24]:
model_pretrained.eval()

MobileNetV2ForImageClassification(
  (mobilenet_v2): MobileNetV2Model(
    (conv_stem): MobileNetV2Stem(
      (first_conv): MobileNetV2ConvLayer(
        (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (conv_3x3): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (reduce_1x1): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (normalization): BatchNorm2d(16, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
      )
    )
    (layer): ModuleList(
      (0): MobileNetV2InvertedResidual(
        (expand_1x1): MobileNe

In [25]:
trainer.evaluate()

{'eval_loss': 0.7992069125175476,
 'eval_accuracy': 0.7357,
 'eval_runtime': 12.9925,
 'eval_samples_per_second': 769.677,
 'eval_steps_per_second': 12.084,
 'epoch': 20.0}

In [26]:
reset_seed(42)

In [30]:
model_pretrained_whole = get_mobilenet()

In [31]:
training_args = get_training_args("./results/cifar10-pretrained", './logs/cifar10-pretrained')

In [32]:
trainer = Trainer(
    model=model_pretrained_whole,
    args=training_args,
    train_dataset=train_dataset_hf,
    eval_dataset=test_dataset_hf,
    compute_metrics=compute_metrics,
)

In [33]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.665700,0.351591,0.881900
2,0.182500,0.456693,0.859000
3,0.109300,0.312467,0.906600
4,0.052500,0.293010,0.915200
5,0.035100,0.563375,0.872000
6,0.016500,0.432066,0.905700
7,0.013400,0.597252,0.876200
8,0.008200,0.704292,0.860900
9,0.005200,0.474817,0.909500
10,0.004600,0.451907,0.917800


TrainOutput(global_step=15640, training_loss=0.048197138210868136, metrics={'train_runtime': 2689.1165, 'train_samples_per_second': 371.869, 'train_steps_per_second': 5.816, 'total_flos': 2.020099608576e+18, 'train_loss': 0.048197138210868136, 'epoch': 20.0})

In [34]:
model_pretrained_whole.eval()

MobileNetV2ForImageClassification(
  (mobilenet_v2): MobileNetV2Model(
    (conv_stem): MobileNetV2Stem(
      (first_conv): MobileNetV2ConvLayer(
        (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (conv_3x3): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (reduce_1x1): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (normalization): BatchNorm2d(16, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
      )
    )
    (layer): ModuleList(
      (0): MobileNetV2InvertedResidual(
        (expand_1x1): MobileNe

In [35]:
trainer.evaluate()

{'eval_loss': 0.41933944821357727,
 'eval_accuracy': 0.9266,
 'eval_runtime': 13.1012,
 'eval_samples_per_second': 763.287,
 'eval_steps_per_second': 11.984,
 'epoch': 20.0}

In [42]:
reset_seed(42)

In [43]:
student_model_pretrained = get_mobilenet()

In [44]:
freeze_model(student_model_pretrained)

In [45]:
training_args = get_training_args("./results/cifar10-pretrained-head-KD", './logs/cifar10-pretrained-head-KD')

In [46]:
trainer = ImageDistilTrainer(
    student_model=student_model_pretrained,
    teacher_model=teacher_model,
    args=training_args,
    train_dataset=train_dataset_hf,
    eval_dataset=test_dataset_hf,
    compute_metrics=compute_metrics,
    temperature = 5,
    lambda_param = 0.5
)

In [47]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.310300,0.962174,0.637000
2,0.838800,0.903210,0.636900
3,0.793800,0.769327,0.712900
4,0.748400,0.747457,0.715100
5,0.739600,0.805402,0.685600
6,0.722300,0.742205,0.720900
7,0.720900,0.832782,0.664400
8,0.710900,0.877281,0.640300
9,0.707600,0.753492,0.705200
10,0.705900,0.750468,0.720900


TrainOutput(global_step=15640, training_loss=0.7418093366696097, metrics={'train_runtime': 11090.4968, 'train_samples_per_second': 90.167, 'train_steps_per_second': 1.41, 'total_flos': 2.020099608576e+18, 'train_loss': 0.7418093366696097, 'epoch': 20.0})

In [48]:
student_model_pretrained.eval()

MobileNetV2ForImageClassification(
  (mobilenet_v2): MobileNetV2Model(
    (conv_stem): MobileNetV2Stem(
      (first_conv): MobileNetV2ConvLayer(
        (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (conv_3x3): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (reduce_1x1): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (normalization): BatchNorm2d(16, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
      )
    )
    (layer): ModuleList(
      (0): MobileNetV2InvertedResidual(
        (expand_1x1): MobileNe

In [49]:
trainer.evaluate()

{'eval_loss': 0.715542197227478,
 'eval_accuracy': 0.737,
 'eval_runtime': 91.5307,
 'eval_samples_per_second': 109.253,
 'eval_steps_per_second': 1.715,
 'epoch': 20.0}

In [50]:
reset_seed(42)

In [51]:
student_model_pretrained_whole = get_mobilenet()

In [52]:
training_args = get_training_args("./results/cifar10-pretrained-KD", './logs/cifar10-pretrained-KD')

In [53]:
trainer = ImageDistilTrainer(
    student_model=student_model_pretrained_whole,
    teacher_model=teacher_model,
    args=training_args,
    train_dataset=train_dataset_hf,
    eval_dataset=test_dataset_hf,
    compute_metrics=compute_metrics,
    temperature = 5,
    lambda_param = 0.5
)

In [54]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.544800,0.346203,0.885200
2,0.229400,0.377549,0.867500
3,0.183900,0.246646,0.921600
4,0.146200,0.229126,0.926500
5,0.131700,0.368331,0.878800
6,0.116000,0.258062,0.915100
7,0.112300,0.340128,0.884500
8,0.104500,0.333817,0.880100
9,0.100700,0.238064,0.920700
10,0.099100,0.213275,0.932400


TrainOutput(global_step=15640, training_loss=0.12985367409103668, metrics={'train_runtime': 12071.5298, 'train_samples_per_second': 82.84, 'train_steps_per_second': 1.296, 'total_flos': 2.020099608576e+18, 'train_loss': 0.12985367409103668, 'epoch': 20.0})

In [55]:
student_model_pretrained.eval()

MobileNetV2ForImageClassification(
  (mobilenet_v2): MobileNetV2Model(
    (conv_stem): MobileNetV2Stem(
      (first_conv): MobileNetV2ConvLayer(
        (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (conv_3x3): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (reduce_1x1): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (normalization): BatchNorm2d(16, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
      )
    )
    (layer): ModuleList(
      (0): MobileNetV2InvertedResidual(
        (expand_1x1): MobileNe

In [57]:
trainer.evaluate()

{'eval_loss': 0.1961217224597931,
 'eval_accuracy': 0.9371,
 'eval_runtime': 91.3167,
 'eval_samples_per_second': 109.509,
 'eval_steps_per_second': 1.719,
 'epoch': 20.0}